In [8]:
#keeping this jsut for reference.

,created_utc,ups,link_id,id,author,body,parent_id,root,depth,num_siblings,num_children,num_comments_author,num_previous_comments,num_later_comments,time_since_root,time_since_parent,num_comments_subtree,height_subtree
0,1430438400,3.0,34f9rh,cqug90j,jesse9o3,No one has a European accent either because i...,cqug2sr,cqug2sr,1,1,0,1,0,0,0.000000,0.000000,1,0
1,1430438400,3.0,34fvry,cqug90k,beltfedshooter,That the kid ..reminds me of Kevin. so sad :-(,34fvry,34fvry,1,1443,0,1,0,3512,0.000000,0.000000,1,0
2,1430438400,5.0,34ffo5,cqug90z,InterimFatGuy,NSFL,cqu80zb,cqu80zb,1,5,0,1,0,11,0.000000,0.000000,1,0
3,1430438401,1.0,34aqsn,cqug91c,JuanTutrego,I'm a guy and I had no idea this was a thing g...,cqtdj4m,cqtdj4m,1,4,0,1,0,4,0.000000,0.000000,1,0
4,1430438401,101.0,34f9rh,cqug91e,dcblackbelt,"Mid twenties male rocking skinny jeans/pants, ...",cquc4rc,cquc4rc,1,2,1,1,0,3,0.000000,0.000000,3,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4234965,1433116795,NaN,37y5rx,crrbelu,monster860,Does the sun set in the west/rise in the east?...,37y5rx,37y5rx,1,1496,0,1,3265,0,12.452222,0.000000,1,0
4234966,1433116795,NaN,37y5rx,crrbelv,jsimo36,Coffee.,37y5rx,37y5rx,1,1496,0,1,3265,0,12.452222,0.000000,1,0
4234967,1433116796,NaN,380jx2,crrbemp,torianicole731,"people who cannot make up their mind, my bulls...",380jx2,380jx2,1,9,0,1,8,0,0.522500,0.000000,1,0
4234968,1433116797,NaN,37yawp,crrbenh,bellypouch,Give them to Irish people in exchange for doin...,37yawp,37yawp,1,1779,0,1,3519,0,11.398056,0.000000,1,0


### **Toulouse School of Economics**
#### **M2 Statistics & Econometrics**
---
## **Web Mining**
### Final Project
---
#### Anh-Dung LE
#### Sherman ALINE
---

## Import necessary packages and setting for multiprocessing 

In [26]:
import pandas as pd
import numpy as np
import string, unidecode, re, os, json
import matplotlib as plt
import networkx as nx
from networkx.algorithms.traversal.depth_first_search import dfs_tree
#from dask.distributed import Client
#import dask.dataframe as dd
from nltk.corpus import stopwords
from multiprocessing import dummy

#core = os.cpu_count() - 2
#client = Client(n_workers = core, threads_per_worker = 1, memory_limit = '25GB')
my_stopwords = stopwords.words('english')
#P = dummy.Pool(processes = core)

## Import dataset

In [59]:
from pathlib import Path
path = Path('/Users/sma/Desktop/LEARNING/M2/S2/PROJ/webmin/comments_students.csv')
df = pd.read_csv(path, header = 0, engine = 'c',
                 usecols = ['created_utc', 'ups', 'link_id', 'id', 'author', 'body', 'parent_id'],
                nrows = 10000000)

In [60]:
df.head()

,created_utc,ups,link_id,id,author,body,parent_id
0,1430438400,3.0,t3_34f9rh,cqug90j,jesse9o3,No one has a European accent either because i...,t1_cqug2sr
1,1430438400,3.0,t3_34fvry,cqug90k,beltfedshooter,That the kid ..reminds me of Kevin. so sad :-(,t3_34fvry
2,1430438400,5.0,t3_34ffo5,cqug90z,InterimFatGuy,NSFL,t1_cqu80zb
3,1430438401,1.0,t3_34aqsn,cqug91c,JuanTutrego,I'm a guy and I had no idea this was a thing g...,t1_cqtdj4m
4,1430438401,101.0,t3_34f9rh,cqug91e,dcblackbelt,"Mid twenties male rocking skinny jeans/pants, ...",t1_cquc4rc


We remove prefix so that columns `parent_id` and `link_id` have the same format as column `id`.

In [61]:
df['parent_id'] = df['parent_id'].str.split('_', expand = True)[1]
df['link_id'] = df['link_id'].str.split('_', expand = True)[1]

`ind` is the indices of  "normal" rows that will be fed into the model later.

In [62]:
ind = (df['body'] != 'deleted') & (df['body'] != '[deleted]') & df['body'].notna()

We save this dataset for later use.

In [ ]:
#THIS PART IS WHERE DUNGS CODES FOR GRAPHS ARE. DELETED FOR EASE OF DEVELOPMENT.
#path = r'E:\\M2 EconStat\\Web Mining\\prefix_removed.csv'
#df.to_csv(path, index = False, encoding =  'utf-8')

Let's summerize $10$ graph features we've extracted so far.

- `depth`:

- `num_siblings`:

- `num_children`:

- `num_comments_author`:

- `num_previous_comments`:

- `num_later_comments`:

- `time_since_root`:

- `time_since_parent`:

- `num_comments_subtree`:

- `height_subtree`:

## Text featurization

As a user of reddit I will apply some domain knowledge in the process of selecting text features. 

* text emotes - 
* emojis - emoticons may be used to detect sentiment. additionally they may signal a overly-casual post which may not be recieved well.

* sentiment in post - funny comments, as well as serious and informative posts will get upvotes. in long comment threads, a hostile tone may get upvotes (for one party, while the other party with hostile tone will get many downvotes)
    * (using TF-IDF) ?
    * BERT ?

* length - long comments may be an indication of high-information or high-effort post

* #punctuation - this is another indicator that a post is likely to be well-written, correlated with useful information and understandable writing etc. all things which will get upvotes on AskReddit. Extremely large amounts of punctuation may also indicate visual text jokes being made using markdown.

* #paragraphs - indicator of high amiount of content/contribution, high effort. 
               - we detect paragraphs by the numbe rof double spaces. doublespaces are removed on reddit, and thourhg manual investigation I have determined that double spaces in our corpora are new lines.

* capital letters

* number of links in a post - citations generally get upvotes, an indicator of sharing information.

* sensitive content: contains 'NSFW' or 'NSFL'

* refer to other users /u/*

* refer to subreditt /r/

* quotes

markdown: https://daringfireball.net/projects/markdown/ https://www.reddit.com/r/reddit.com/comments/6ewgt/reddit_markdown_primer_or_how_do_you_do_all_that/

In [ ]:
#unparse html entities
#-remove rows where body is NA.
#- extract emojis, extract emoticons
#- remove them ^^
#- get number of links & remove them
#- get length, number of punctuation, capital letters, num paragraphs (by finding \n characters)
#- run text cleaning
#- sentiment analysis shit.

# - &gt; is > which is used for quotes.

In [ ]:
import HTMLParser
# TODO &lt; &gt; probably others...

# '   - ' bullet points (with a newline but that always works that way.)
# '  ' new line.

In [72]:
#df_text = df.dropna(subset=['body'])

In [179]:
#df_text.loc[df.body.str.contains('paragraphs', na=False)]
df_text.loc[df_text.id=='crqm50x'].body[4145696]

"Two things:  1) surely Word of God is very easy countered with Barthes' [Death of the Author](http://en.wikipedia.org/wiki/Death_of_the_Author) ?  2) I write for a living, I know what Word of God is. That's not what I'm talking about. I'm a gay woman, and if I was going to write a gay character they'd be gay. No need for me to say months after the book has sold millions of copies.  They'd just be gay. What JK is doing is trying to have her cake and eat it; she didn't write a gay character (or at least, she hinted so distantly at it that it wasn't possible to deduce without her say so) but she wants the *credit* for writing one and being progressive. So she says he's gay outside of the books canon and thus is completely meaningless to me and many other people. I understand that in fandom circles, it's fine for her to do that and in terms of x character crushes on y character it is. In terms of this, though, it's not. After she's sold millions of books to evangelicals, to homophobic nat

In [ ]:
#ಠ
#i think paraghraphs arent retained

In [86]:
import marko 

In [157]:
marko.convert("I quit   What?!    [You have no idea, how high I can fly](https://www.youtube.com/watch?v=VYCehJk7UDE)    That scene popped up when you said >and get repeatedly overlooked by corporate despite performing well...    Michael seems to get the shit end of stick even though he's just a good guy, a bit of a goof, but still just a genuine good guy")

'<p>I quit   What?!    <a href="https://www.youtube.com/watch?v=VYCehJk7UDE">You have no idea, how high I can fly</a>    That scene popped up when you said &gt;and get repeatedly overlooked by corporate despite performing well...    Michael seems to get the shit end of stick even though he\'s just a good guy, a bit of a goof, but still just a genuine good guy</p>\n'

In [147]:
dir(temp)
#temp

['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_prefix',
 'children',
 'inline_children',
 'link_ref_defs',
 'match',
 'override',
 'parse',
 'parse_inline',
 'priority',
 'virtual']

In [156]:
#len(temp.children)
temp.override

False

In [ ]:

#emoji.

## Text cleaning

In [23]:
# TODO: write text cleaning function which actually works.

#def text_cleaning(df, colname, ind):
#    
#    tmp = df[colname][ind].copy()
#    
#    # Convert text to lowercase
#    tmp = tmp.str.lower()
#    
#    # Delete punctuation
#    tmp = tmp.str.replace('\n', ' ')
#    tmp = tmp.str.replace('\r', ' ')    
#    tmp = tmp.str.replace(r"((?!{}).)".format('(\\b[-/]\\b|[a-zA-Z0-9])'), ' ', regex = True)
#    
#    # Tokenize
#    tmp = tmp.str.split()
#    
#    # Delete stop words
#    tmp = tmp.apply(lambda x: [w for w in x if w not in my_stopwords])
#    
#    # Reverse tokenize
#    df.loc[ind, colname] = tmp.map(lambda word: ' '.join(word))
#    
#
#df2 = text_cleaning(df, 'body', ind)

## TF-IDF

In [24]:
df.head()

AttributeError: 'NoneType' object has no attribute 'head'

In [15]:
data = df['body']
y = df['ups']

n_feature = 50

tfi_df_vec = TfidfVectorizer(use_idf = True,
                             max_features = n_feature)

X = tfi_df_vec.fit_transform(data)

TypeError: 'NoneType' object is not subscriptable

## Split dataset into test and train sets

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                    test_size = 0.33,
                                                    random_state = 42)

## Random forest model

In [ ]:
model = RandomForestRegressor(n_jobs = -1)
model.fit(X_train, y_train)

# Get the mean absolute error on the validation data
y_pred = model.predict(X_test)
MAE = mean_absolute_error(y_test , y_pred)
print('Random forest validation MAE = ', MAE)

## XGBoost model

In [ ]:
# from xgboost import XGBRegressor
# XGBModel = XGBRegressor()
# XGBModel.fit(X_train, y_train , verbose = False)

# # Get the mean absolute error on the validation data :
# XGBpredictions = XGBModel.predict(X_test)
# MAE = mean_absolute_error(y_test , XGBpredictions)
# print('XGBoost validation MAE = ', MAE)

## Preliminary deep neural network

In [ ]:
# model = Sequential()
# model.add(Dense(1000, input_dim = 1000, activation = 'relu', kernel_initializer='normal'))
# model.add(Dense(8, activation = 'relu', kernel_initializer='normal'))
# model.add(Dense(1, activation = 'linear', kernel_initializer='normal'))
# model.compile(loss = 'mean_absolute_error',
#               optimizer = 'adam',
#               metrics = ['accuracy'])
# print(model.summary())
# model.fit(X_train, y_train,
#           epochs = 3,
#           batch_size = 10,
#           validation_data = (X_test, y_test),
#           verbose = 1)